# Tasking Black Sky - build tasking parameters and place an order

This notebook proposes a method for building tasking parameters needed for placing a Black Sky tasking order. A similar approach can be used with other collections, the only variation would be around the number required parameters.

To place a tasking order we need to provide tasking parameters in the form of a json file. Here we will build a tasking parameters json file using the product schema as our foundation.


In [1]:
import up42
from pathlib import Path
import json

## Authenticate

Each API call is made at a project level. To find project credentials go to Projects and select an existing project or create a new one.

Go to the Developers tab in the project and copy the values of Project API Key and Project ID.

Don't share your credentials with others. They allow anyone to access your project and consume the UP42 credits associated with your account. If your credentials were compromised, generate a new API key. If needed more info can be found on our documentations page.

Past these credentials into './proj_conf_file.json' located in this directory. 

In [2]:
proj_config_file = Path.cwd()/'proj_conf_file.json'
up42.authenticate(proj_config_file)

2023-05-10 16:31:21,915 - Got credentials from config file.
2023-05-10 16:31:23,089 - Authentication with UP42 successful!


## AOI
For BLACK SKY data a "Point" geometry must be used

In [3]:
aoi = up42.read_vector_file(Path.cwd()/'aoi/aoi_blacksky_point.geojson')
aoi['features'][0]['geometry']

{'type': 'Point', 'coordinates': (-80.84239741821287, 35.23310416169144)}

### View tasking products

In [4]:
tasking = up42.initialize_tasking()
products = tasking.get_data_products(basic=True)

In [5]:
for prod in products:
    print(f"{prod} -> {products[prod]['data_products']} \n")

SPOT 6/7 Tasking -> {'Custom': '3a89d24e-515a-460f-a494-96be55da10a9', 'Analytic': '9d16c506-a6c0-4cf2-a020-8ecaf10b4160', 'Display': '47a693ba-4f8b-414d-8d5b-697373df4765'} 

Pléiades Neo Tasking -> {'Custom': '07c33a51-94b9-4509-84df-e9c13ea92b84', 'Analytic (Mono)': '123eabab-0511-4f36-883a-80928716c3db', 'Display (Mono)': '469f9b2f-1631-4c09-b66d-575abd41dc8f'} 

Capella Space Tasking -> {'Custom': '83e21b35-e431-43a0-a1c4-22a6ad313911'} 

Pléiades Tasking -> {'Custom': '4f866cd3-d816-4c98-ace3-e6105623cf13', 'Analytic': 'bd102407-1814-4f92-8b5a-7697b7a73f5a', 'Display': '28d4a077-6620-4ab5-9a03-c96bf622457e'} 

TerraSAR Tasking -> {'Custom': 'a6f64332-3148-4e05-a475-45a02176f210'} 

BlackSky Tasking -> {'Custom': '613ad1f5-4148-4460-a316-1a97e46058f9'} 

ICEYE Tasking -> {'Custom': '205fd9e1-4b00-4f0a-aabd-01e2f3f4dfba'} 

DailyVision Tasking -> {'Custom': '19a60197-2738-47bf-83ef-1b780fa142c8'} 

EarthScanner Tasking -> {'Custom': '8010965f-aa97-41ee-bd4e-f6355f0967da'} 

SuperVi

### Build tasking parameters

From the options printed above we select BlackSky Tasking and copy the id under 'Custom'

In [6]:
bsk_tasking_id = "613ad1f5-4148-4460-a316-1a97e46058f9"

Let's pull up the full schema for this tasking product, the schema provides all the information needed i.e. parameters and possible values

In [7]:
tasking_product_schema = tasking.get_data_product_schema(bsk_tasking_id)
# tasking_product_schema

In [8]:
tasking_product_schema['properties'].keys()

dict_keys(['geometry', 'displayName', 'extraDescription', 'acquisitionStart', 'acquisitionEnd', 'acquisitionMode', 'cloudCoverage', 'incidenceAngle', 'priority'])

In [9]:
params_payload = tasking_product_schema['properties']
params_payload

{'geometry': {'title': 'Geometry',
  'type': 'object',
  'allOf': [{'$ref': '#/definitions/Point'}]},
 'displayName': {'title': 'Order Name', 'type': 'string'},
 'extraDescription': {'title': 'Description', 'type': 'string'},
 'acquisitionStart': {'title': 'Start',
  'type': 'string',
  'format': 'date-time'},
 'acquisitionEnd': {'title': 'End', 'type': 'string', 'format': 'date-time'},
 'acquisitionMode': {'title': 'Acquisition Mode',
  'description': 'Standard mode acquires single image scene in daylight with standard product specifications. Burst collects multiple images in rapid sequence for motion analysis. Area Coverage collects mulitple images over an area larger than a single image scene. Stereo collects multiple images from different viewing angles for 3D visualization and analysis.',
  'type': 'string',
  'allOf': [{'$ref': '#/definitions/acquisitionMode'}]},
 'cloudCoverage': {'title': 'Max Cloud Coverage (%)',
  'description': 'Maximum allowed cloud coverage in percentage. 

We can start putting our parameters together. The json payload we are looking to build is essentially a dictionary composed of two keys: “dataProduct” and “params”

Let's create new dictionary

In [10]:
params_build = {"dataProduct":"",
                    "params":{}
                   }

We identified the “dataProduct” id previously and passed it to a variable, we can use this as the value to the "dataProduct" key for our dictionary.
We can also pass the params_payload we retrieved from the product schema earlier as our temporary parameters. We will update these with actual parameters soon.


In [11]:
params_build["dataProduct"] = bsk_tasking_id
params_build["params"] = params_payload

Sanity check, let's see what this looks like

In [12]:
params_build

{'dataProduct': '613ad1f5-4148-4460-a316-1a97e46058f9',
 'params': {'geometry': {'title': 'Geometry',
   'type': 'object',
   'allOf': [{'$ref': '#/definitions/Point'}]},
  'displayName': {'title': 'Order Name', 'type': 'string'},
  'extraDescription': {'title': 'Description', 'type': 'string'},
  'acquisitionStart': {'title': 'Start',
   'type': 'string',
   'format': 'date-time'},
  'acquisitionEnd': {'title': 'End', 'type': 'string', 'format': 'date-time'},
  'acquisitionMode': {'title': 'Acquisition Mode',
   'description': 'Standard mode acquires single image scene in daylight with standard product specifications. Burst collects multiple images in rapid sequence for motion analysis. Area Coverage collects mulitple images over an area larger than a single image scene. Stereo collects multiple images from different viewing angles for 3D visualization and analysis.',
   'type': 'string',
   'allOf': [{'$ref': '#/definitions/acquisitionMode'}]},
  'cloudCoverage': {'title': 'Max Cloud

If all went well you should have a dictionary with a lot of info but you can use it to discover the values you are allowed to pass. Let's view the actual keys and see what needs to be updated again.  

In [14]:
params_build["params"].keys()

dict_keys(['geometry', 'displayName', 'extraDescription', 'acquisitionStart', 'acquisitionEnd', 'acquisitionMode', 'cloudCoverage', 'incidenceAngle', 'priority'])

Cool, so these are the keys and therefore tasking parameters we need to provide.
In the next code block we assign each parameter value to a variable and will update our dictionary 


In [15]:
# Provide your tasking parameters here

displayName = "TEST_ALEX_BSK"
extraDescription = "SDK build demo"
acquisitionStart = "2023-05-20"
acquisitionEnd = "2023-06-20"
acquisitionMode = "standard"
cloudCoverage = 30
incidenceAngle = 20
priority = "standard"

# Make the necessary modfication in the dictionary

params_build["params"]["geometry"] = aoi['features'][0]['geometry']
params_build["params"]["displayName"] = displayName
params_build["params"]["extraDescription"] = extraDescription
params_build["params"]["acquisitionStart"] = str(acquisitionStart)+"T00:00:00.000Z"
params_build["params"]["acquisitionEnd"] = str(acquisitionEnd)+"T00:00:00.000Z"
params_build["params"]["acquisitionMode"] = acquisitionMode
params_build["params"]["cloudCoverage"] = cloudCoverage
params_build["params"]["incidenceAngle"] = incidenceAngle
params_build["params"]["priority"] = priority

We did ! Let's look at the json payload we put together containing our Black Sky tasking parameters.

In [17]:
params_build

{'dataProduct': '613ad1f5-4148-4460-a316-1a97e46058f9',
 'params': {'geometry': {'type': 'Point',
   'coordinates': (-80.84239741821287, 35.23310416169144)},
  'displayName': 'TEST_ALEX_BSK',
  'extraDescription': 'SDK build demo',
  'acquisitionStart': '2023-05-20T00:00:00.000Z',
  'acquisitionEnd': '2023-06-20T00:00:00.000Z',
  'acquisitionMode': 'standard',
  'cloudCoverage': 30,
  'incidenceAngle': 20,
  'priority': 'standard'}}

For reference and perhaps for future use let's save these order parameters to a json file.

In [18]:
with open('tasking_parameters_payload.json' , 'w') as tsk:
    json.dump(params_build, tsk, indent = 4)

### Place the order

There are no costs at this point, a feasibility study is triggered and a quote will be made available in one to three business days depending on the complexity of the order. You are charged and the order is activated only after you have accepted the quote that will be made available to you.

In [19]:
order = tasking.place_order(params_build)
order

2023-05-10 16:36:41,427 - Order is PLACED
2023-05-10 16:36:41,429 - Order 61ca0745-5895-4df0-87bd-364fe5b4f334 is now PLACED.


Order(order_id: 61ca0745-5895-4df0-87bd-364fe5b4f334, assets: [], dataProvider: None, status: PLACED, createdAt: 2023-05-10T20:36:40.311928Z, updatedAt: 2023-05-10T20:36:40.311928Z)

Happy tasking!!

If you have comments or questions please reach out to alex.bishop@up42.com

Documentation


https://docs.up42.com/data

https://docs.up42.com/developers/api#tag/credit-packages/operation/getCreditPack

https://sdk.up42.com/